In [1]:
import time
import datetime
from collections import OrderedDict
import pandas as pd
import numpy as np
import scipy
from xgboost import XGBRegressor
import sklearn
from sklearn.model_selection import ShuffleSplit, KFold, RepeatedKFold, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer, mean_squared_error
from sklearn.datasets import load_boston
from sklearn.base import BaseEstimator, RegressorMixin

In [2]:
N_JOBS = 16
datasets = OrderedDict()
results = OrderedDict()
results2 = OrderedDict()
results_gs = OrderedDict()
results_gs2 = OrderedDict()
results_gs3 = OrderedDict()

In [3]:
datasets['boston'] = (load_boston()['data'], load_boston()['target'])

In [4]:
df = pd.read_csv('datasets/concrete_data.csv')
datasets['concrete'] =  (df.iloc[:, :-1], df.iloc[:, -1])

In [5]:
df = pd.read_csv('datasets/energy_efficiency.csv')
datasets['energy'] =  (df.iloc[:, :-2], df.iloc[:, -2])

In [6]:
df = pd.read_csv('datasets/kin8nm.csv')
datasets['kin8nm'] =  (df.iloc[:, :-1], df.iloc[:, -1])

In [7]:
df = pd.read_table('datasets/naval.txt', sep='\s+', header=None)
datasets['naval'] =  (df.iloc[:, :-2], df.iloc[:, -2])

In [8]:
df = pd.read_csv('datasets/power.csv')
datasets['power'] =  (df.iloc[:, :-1], df.iloc[:, -1])

In [9]:
df = pd.read_csv('datasets/protein.csv')
datasets['protein'] =  (df.iloc[:, 1:], df.iloc[:, 0])

In [10]:
df = pd.read_csv('datasets/wine.csv', sep=';')
datasets['wine'] =  (df.iloc[:, :-1], df.iloc[:, -1])

In [11]:
df = pd.read_table('datasets/yacht.txt', sep='\s+', header=None)
datasets['yacht'] =  (df.iloc[:, :-1], df.iloc[:, -1])

In [12]:
#df = pd.read_table('datasets/year.txt', sep=',', header=None)
#datasets['year'] =  (df.iloc[:, 1:], df.iloc[:, 0])

In [16]:
def normal_ll(y, y_pred):
    mean, log_var = y_pred
    return scipy.stats.norm.logpdf(y, loc = mean, scale = np.exp(log_var/2)).mean()

ll_scorer = make_scorer(normal_ll, greater_is_better=True)

In [17]:
def ll_objective(y_true, y_pred):
    err = y_true
    log_var = y_pred
    #grad = 1/(2*np.exp(log_var)) - 1/(2*np.exp(2*log_var))*(err**2)
    grad = -1/(2*np.exp(log_var))*(1 - 1/np.exp(log_var)*(err**2))
    #hess = -1/(2*np.exp(2*log_var)) + 2/(2*np.exp(3*log_var))*(err**2)
    hess = 1/np.exp(1.5*log_var) - 2/np.exp(2.5*log_var)*(err**2)
    return grad, hess


class XGBLogLikelihood(BaseEstimator, RegressorMixin):  
    
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3, subsample=1):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.subsample = subsample

    def fit(self, X, y):
        self.xgb_mean = XGBRegressor(n_estimators=self.n_estimators, learning_rate=self.learning_rate, max_depth=self.max_depth, subsample=self.subsample)
        self.xgb_var = XGBRegressor(objective=ll_objective, n_estimators=self.n_estimators, learning_rate=self.learning_rate, max_depth=self.max_depth, subsample=self.subsample)
        self.xgb_mean.fit(X, y)
        mean = self.xgb_mean.predict(X)
        self.xgb_var.fit(X, y-mean)
        return self

    def predict(self, X, y=None):
        pred_mean = self.xgb_mean.predict(X)
        pred_var = self.xgb_var.predict(X)
        return pred_mean, pred_var

In [18]:
for d, (X,y) in datasets.items():
    reg = XGBRegressor(n_estimators=300)
    if d == 'year':
        cv = ShuffleSplit(3, test_size=0.1)
    elif d == 'protein':
        cv = KFold(n_splits=10)
    else:
        cv = RepeatedKFold(n_splits=10, n_repeats=4)

    rmse = np.sqrt(cross_val_score(reg, X, y, cv=cv, scoring=make_scorer(mean_squared_error), n_jobs=N_JOBS))
    
    ll = cross_val_score(XGBLogLikelihood(n_estimators=300), X, y, cv=cv, scoring=ll_scorer, n_jobs=N_JOBS)
    
    results[d] = (datetime.datetime.now(), d, X.shape[0], X.shape[1],
                  rmse.mean(), scipy.stats.sem(rmse),
                  ll.mean(), scipy.stats.sem(ll))
    
    print(*results[d])

2018-02-19 10:52:54.422334 boston 506 13 2.9420028243079654 0.10706328471131453 -3.04809177380731 0.0060837527038590175
2018-02-19 10:52:57.389770 concrete 1030 8 4.262673935227591 0.06766694211997114 -3.3006479250990255 0.00433244934421644
2018-02-19 10:52:59.830729 energy 768 8 0.39645298556216857 0.009424240576023942 -3.112855006613991 0.0003513882743818383
2018-02-19 10:53:22.141830 kin8nm 8192 8 0.16336162704398388 0.0007048988342431662 -4.01518212417692 0.01936367202859153
2018-02-19 10:54:01.751254 naval 11934 16 0.0018241725684976112 8.808601907480012e-06 -4.206412637310278 2.0135751562973134e-06
2018-02-19 10:54:14.632299 power 9568 4 3.5569447931791673 0.02851355268496357 -4.134097370944497 0.0019604854311048225
2018-02-19 10:54:57.785823 protein 45730 9 4.369270468925696 0.016002545621752278 -4.58736556447999 0.03649842968374891
2018-02-19 10:55:01.650544 wine 1599 11 0.6061434488748418 0.0068340070551303825 -3.4594003228884143 0.0004347115591736016
2018-02-19 10:55:03.44960

In [19]:
for i, j in results.items():
    res = ['{:.2f}'.format(x) for x in j[-4:]]
    print(i, 'rmse', ' & ' + ' \\tpm '.join(res[0:2]))
    print(i, 'll', ' & ' + ' \\tpm '.join(res[2:]))

boston rmse  & 2.94 \tpm 0.11
boston ll  & -3.05 \tpm 0.01
concrete rmse  & 4.26 \tpm 0.07
concrete ll  & -3.30 \tpm 0.00
energy rmse  & 0.40 \tpm 0.01
energy ll  & -3.11 \tpm 0.00
kin8nm rmse  & 0.16 \tpm 0.00
kin8nm ll  & -4.02 \tpm 0.02
naval rmse  & 0.00 \tpm 0.00
naval ll  & -4.21 \tpm 0.00
power rmse  & 3.56 \tpm 0.03
power ll  & -4.13 \tpm 0.00
protein rmse  & 4.37 \tpm 0.02
protein ll  & -4.59 \tpm 0.04
wine rmse  & 0.61 \tpm 0.01
wine ll  & -3.46 \tpm 0.00
yacht rmse  & 0.56 \tpm 0.04
yacht ll  & -2.85 \tpm 0.00


In [20]:
parameters = {'max_depth': [2,3,4], 
              'n_estimators':[100, 250, 500], 
              'learning_rate':[0.01, 0.1, 0.25],
              'subsample': [0.5, 1.0]}

for d, (X,y) in datasets.items():
    if d == 'year':
        cv = ShuffleSplit(3, test_size=0.1)
    elif d == 'protein':
        cv = KFold(n_splits=10)
    else:
        cv = RepeatedKFold(n_splits=10, n_repeats=4)
    
    clf = GridSearchCV(XGBLogLikelihood(), parameters, scoring=ll_scorer, cv=cv, n_jobs=N_JOBS)
    clf.fit(X, y)
        
    results_gs[d] = (datetime.datetime.now(), clf.best_score_, clf.best_params_)
    
    print(*results_gs[d])

2018-02-19 11:07:06.936343 -2.8560392205442238 {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500, 'subsample': 0.5}
2018-02-19 11:08:16.484149 -3.1081573651515066 {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 250, 'subsample': 0.5}
2018-02-19 11:09:09.768625 -2.862056525695023 {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 250, 'subsample': 0.5}
2018-02-19 11:18:49.643194 -1.5716992556395888 {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 100, 'subsample': 0.5}
2018-02-19 11:35:38.971516 -1.5654163327805022 {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 100, 'subsample': 0.5}
2018-02-19 11:41:48.592901 -3.7067032195380545 {'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 500, 'subsample': 0.5}
2018-02-19 11:57:46.988752 -4.2330589496370745 {'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 250, 'subsample': 0.5}
2018-02-19 11:59:31.716025 -2.1326860344399727 {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample':

In [21]:
def ll_objective2(y_true, y_pred):
    err = y_true
    log_var = y_pred
    #grad = 1/(2*np.exp(log_var)) - 1/(2*np.exp(2*log_var))*(err**2)
    grad = -1/(2*np.exp(log_var))*(1 - 1/np.exp(log_var)*(err**2))
    #hess = -1/(2*np.exp(2*log_var)) + 2/(2*np.exp(3*log_var))*(err**2)
    hess = 1/np.exp(1.5*log_var) - 2/np.exp(2.5*log_var)*(err**2)
    return -1*grad, -1*hess


class XGBLogLikelihood2(BaseEstimator, RegressorMixin):  
    
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3, subsample=1):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.subsample = subsample

    def fit(self, X, y):
        self.xgb_mean = XGBRegressor(n_estimators=self.n_estimators, learning_rate=self.learning_rate, max_depth=self.max_depth, subsample=self.subsample)
        self.xgb_var = XGBRegressor(objective=ll_objective2, n_estimators=self.n_estimators, learning_rate=self.learning_rate, max_depth=self.max_depth, subsample=self.subsample)
        self.xgb_mean.fit(X, y)
        mean = self.xgb_mean.predict(X)
        self.xgb_var.fit(X, y-mean)
        return self

    def predict(self, X, y=None):
        pred_mean = self.xgb_mean.predict(X)
        pred_var = self.xgb_var.predict(X)
        return pred_mean, pred_var

In [22]:
for d, (X,y) in datasets.items():
    reg = XGBRegressor(n_estimators=300)
    if d == 'year':
        cv = ShuffleSplit(3, test_size=0.1)
    elif d == 'protein':
        cv = KFold(n_splits=10)
    else:
        cv = RepeatedKFold(n_splits=10, n_repeats=4)

    rmse = np.sqrt(cross_val_score(reg, X, y, cv=cv, scoring=make_scorer(mean_squared_error), n_jobs=N_JOBS))
    
    ll = cross_val_score(XGBLogLikelihood2(n_estimators=300), X, y, cv=cv, scoring=ll_scorer, n_jobs=N_JOBS)
    
    results2[d] = (datetime.datetime.now(), d, X.shape[0], X.shape[1],
                  rmse.mean(), scipy.stats.sem(rmse),
                  ll.mean(), scipy.stats.sem(ll))
    
    print(*results2[d])

2018-02-19 12:00:09.332102 boston 506 13 2.9288799303959387 0.10443712481028865 -5.27028103982162 0.45934299974521114
2018-02-19 12:00:12.375992 concrete 1030 8 4.4208130189723835 0.10621084584134176 -3.000308708610759 0.12785090762825235
2018-02-19 12:00:14.877895 energy 768 8 0.3953540194205101 0.009448205614016034 -0.9293331632462923 0.026894157103794572
2018-02-19 12:00:36.738400 kin8nm 8192 8 0.16342703660758953 0.0005816751956679289 -1.1769789352021471 7.830435246564618e-05
2018-02-19 12:01:13.781969 naval 11934 16 0.001825235063318341 9.220522528133072e-06 -1.168939547060359 1.0320206019638962e-08
2018-02-19 12:01:29.042958 power 9568 4 3.5485108961808933 0.03251891187712185 -2.965539034421579 0.015074777768667592
2018-02-19 12:02:16.029305 protein 45730 9 4.369270468925696 0.016002545621752278 -3.1276395391124443 0.003930987440531903
2018-02-19 12:02:20.456021 wine 1599 11 0.6068884528204511 0.006007793884211449 -0.997959578011498 0.015496887013273924
2018-02-19 12:02:22.282156

In [23]:
parameters = {'max_depth': [2,3,4], 
              'n_estimators':[100, 250, 500], 
              'learning_rate':[0.01, 0.1, 0.25],
              'subsample': [0.5, 1.0]}

for d, (X,y) in datasets.items():
    if d == 'year':
        cv = ShuffleSplit(3, test_size=0.1)
    elif d == 'protein':
        cv = KFold(n_splits=10)
    else:
        cv = RepeatedKFold(n_splits=10, n_repeats=4)
    
    clf = GridSearchCV(XGBLogLikelihood2(), parameters, scoring=ll_scorer, cv=cv, n_jobs=N_JOBS)
    clf.fit(X, y)
        
    results_gs2[d] = (datetime.datetime.now(), clf.best_score_, clf.best_params_)
    
    print(*results_gs2[d])

2018-02-19 12:03:31.917538 -2.6138184191032146 {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 100, 'subsample': 0.5}
2018-02-19 12:04:54.036943 -2.8484179769586104 {'learning_rate': 0.25, 'max_depth': 2, 'n_estimators': 500, 'subsample': 0.5}
2018-02-19 12:05:50.339177 -0.900907103201415 {'learning_rate': 0.25, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
2018-02-19 12:14:46.624465 -1.1743100364458898 {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 500, 'subsample': 0.5}
2018-02-19 12:30:04.870825 -1.1689388157845524 {'learning_rate': 0.25, 'max_depth': 4, 'n_estimators': 500, 'subsample': 0.5}
2018-02-19 12:37:36.619553 -2.685051730800071 {'learning_rate': 0.25, 'max_depth': 4, 'n_estimators': 500, 'subsample': 1.0}
2018-02-19 12:59:04.309865 -2.9460863509779065 {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 500, 'subsample': 0.5}
2018-02-19 13:00:59.119451 -0.9700302438258261 {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'subsample': 0.

In [ ]:
parameters = {'max_depth': [2,3,4], 
              'n_estimators':[100, 250, 500], 
              'learning_rate':[0.01, 0.1, 0.25],
              'subsample': [0.5, 1.0]}

def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

rmse_scorer = make_scorer(rmse, greater_is_better=False)

for d, (X,y) in datasets.items():
    if d == 'year':
        cv = ShuffleSplit(3, test_size=0.1)
    elif d == 'protein':
        cv = KFold(n_splits=10)
    else:
        cv = RepeatedKFold(n_splits=10, n_repeats=4)
    
    clf = GridSearchCV(XGBRegressor(), parameters, scoring=rmse_scorer, cv=cv, n_jobs=N_JOBS)
    clf.fit(X, y)
        
    results_gs3[d] = (datetime.datetime.now(), clf.best_score_, clf.best_params_)
    
    print(*results_gs3[d])

2018-02-19 17:46:52.431951 -2.9059940192186784 {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 500, 'subsample': 1.0}
2018-02-19 17:47:31.413679 -3.8078824079326603 {'learning_rate': 0.25, 'max_depth': 4, 'n_estimators': 500, 'subsample': 1.0}
2018-02-19 17:47:56.294140 -0.3135142612618605 {'learning_rate': 0.25, 'max_depth': 4, 'n_estimators': 500, 'subsample': 1.0}
2018-02-19 17:54:46.708426 -0.1327761274431461 {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 500, 'subsample': 0.5}
2018-02-19 18:06:22.413116 -0.0009602129558026635 {'learning_rate': 0.25, 'max_depth': 4, 'n_estimators': 500, 'subsample': 0.5}
2018-02-19 18:10:04.478550 -3.016242265314139 {'learning_rate': 0.25, 'max_depth': 4, 'n_estimators': 500, 'subsample': 1.0}
2018-02-19 18:21:26.015127 -3.875866631700612 {'learning_rate': 0.25, 'max_depth': 4, 'n_estimators': 500, 'subsample': 1.0}
